In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

ModuleNotFoundError: No module named 'sklearn'

In [67]:
df=pd.read_csv(r'C:\Users\rajak\OneDrive\Documents\DATA\100-days-of-machine-learning\day28-column-transformer\covid_toy.csv')

In [68]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [69]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [70]:
X=df.drop(columns=['has_covid'])
y=df.has_covid

In [71]:
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [72]:
X_train

,age,gender,fever,cough,city
35,82,Female,102.0,Strong,Bangalore
11,65,Female,98.0,Mild,Mumbai
84,69,Female,98.0,Strong,Mumbai
44,20,Male,102.0,Strong,Delhi
73,34,Male,98.0,Strong,Kolkata
...,...,...,...,...,...
43,22,Female,99.0,Mild,Bangalore
22,71,Female,98.0,Strong,Kolkata
72,83,Female,101.0,Mild,Kolkata
15,70,Male,103.0,Strong,Kolkata


In [73]:
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_test_age

array([[17],
       [15],
       [71],
       [13],
       [69],
       [80],
       [42],
       [33],
       [16],
       [64],
       [10],
       [82],
       [80],
       [51],
       [60],
       [73],
       [31],
       [34],
       [81],
       [48]], dtype=int64)

# Without Column Transformer

In [74]:
si=SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit(X_test[['fever']])
X_train_fever.shape

(80, 1)

In [75]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.transform(X_test[['cough']])
X_test_cough.shape

(20, 1)

In [76]:
#One HotEncoding on gender and city
ohe=OneHotEncoder(drop='first',sparse=False)
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city=ohe.transform(X_test[['gender','city']])
X_test_gender_city.shape

(20, 4)

In [77]:
X_train_transformed=np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

In [82]:
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)
#This error reason hasn't found

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)

In [79]:
X_train_transformed.shape

(80, 7)

# With column_tranfer

Each tuple expects 3 comma-separated values: first, the name of the transformer, 
which can be practically anything (passed as a string), second is the estimator object,
and the final one being the columns upon which we wish to perform that operation.

In [83]:
from sklearn.compose import ColumnTransformer

In [86]:
transformer=ColumnTransformer(transformers=[
    ('trf1',SimpleImputer(strategy='mean'),['fever']),
    ('trf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('trf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [87]:
transformer.fit_transform(X_train).shape

(80, 7)

In [95]:
X_=transformer.transform(X_test)

In [93]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('trf1', SimpleImputer(), ['fever']),
                                ('trf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('trf3',
                                 OneHotEncoder(drop='first', sparse=False),
                                 ['gender', 'city'])])

In [99]:
pd.DataFrame(X_).isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64

In [100]:
xt=transformer.fit_transform(X_train)

In [101]:
pd.DataFrame(xt).isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64